In [1]:
input_path: str = "starship-olm-1024-stipple.tsp"

# Read the TSP file and conver the coordiantes into a list of tuples
lines: list[str] = []
with open(input_path) as f:
    lines = f.readlines()

# Remove the first lines of description
for i, line in enumerate(lines):
    if line[0] in "0123456789":
        lines = lines[i:]
        break

# Remove line number
points = [line.split()[1:] for line in lines]

# Convert to float tuples
points = [(float(x), float(y)) for x, y in points]

In [2]:
solution_file: str = "test1.cyc"

# Read the cyc file
lines: list[str] = []
with open(solution_file) as f:
    lines = f.readlines()

solution: list[int] = [int(line) for line in lines[1:]]

In [6]:
%pip install mecode

from mecode import GMatrix

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 23.0 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Helpers (copied)

In [7]:
def pen_down(g: GMatrix) -> None:
    """Lower the pen.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
    """

    # This represents lowering the pen, but does not have any real effect as this plotter only prints in 2D.
    if g.is_relative:
        g.move(z=-3.3)
    else:
        g.move(x=g.current_position['x'], y=g.current_position['y'], z=0)

    # This command actually lowers the pen.
    g.write("M9")


def pen_up(g: GMatrix) -> None:
    """Raise the pen.

    Args:
        g (GMatrix): GMatrix instance to use for GCode generation.
    """

    # This represents raising the pen, but does not have any real effect as this plotter only prints in 2D.
    if g.is_relative:
        g.move(z=3.3)
    else:
        g.move(x=g.current_position['x'], y=g.current_position['y'], z=3.3)

    # This command actually raises the pen.
    g.write("M8")


def get_gmatrix(outfile: str = None) -> GMatrix:
    """Create a GMatrix instance and write the starting GCode.

    Args:
        outfile (str, optional): Path to the output file. Defaults to None.

    Returns:
        GMatrix: GMatrix instance.
    """

    start_code = "G91 ;relative\nG21 ;mm\nG92 X0 Y0 Z0 ;reset origin\nF2000;motion speed"
    g = GMatrix(outfile=outfile)
    g.push_matrix()
    g.write(start_code)
    return g


In [8]:
# Create a GMatrix instance and write the starting GCode
g = get_gmatrix("test.gcode")

# Move the pen to a reasonable starting position
pen_up(g)
g.absolute()

# Start drawing
pen_down(g)
for item in solution:
    x, y = points[item]
    g.move(x, y)

# Finish drawing
pen_up(g)